# Getting Started

Go to the setupTable and input your unique token

DO NOT USE MY KEY (Blake)

## Shift enter through initialize

In [6]:
import requests
import time
import json

In [7]:
class Stack():
    def __init__(self):
        self.stack = []
    def push(self, value):
        self.stack.append(value)
    def pop(self):
        if self.size() > 0:
            return self.stack.pop()
        else:
            return None
    def size(self):
        return len(self.stack)

In [8]:
class Queue():
    def __init__(self):
        self.queue = []
    def enqueue(self, value):
        self.queue.append(value)
    def dequeue(self):
        if self.size() > 0:
            return self.queue.pop(0)
        else:
            return None
    def size(self):
        return len(self.queue)

In [9]:
# Import map and room info
with open('worldMap.json', 'r') as f:
    worldMap = json.load(f)
    
with open('worldRoomIndex.json', 'r') as f:
    worldRoomIndex = json.load(f)

In [10]:
# Utils
inverseMap = {'n':'s', 's':'n', 'e':'w', 'w':'e'}

## Input unique token key here

In [11]:
setupTable = {
    "url": 'https://lambda-treasure-hunt.herokuapp.com/api/',
    "token": "d2679616e63cea70c89ff019addc2677a27c9388",
    "init": 'adv/init/',
    "move": 'adv/move/',
    "take": 'adv/take/',
    "sell": 'adv/sell/',
    "drop": 'adv/drop/',
    "checkStatus": 'adv/status/',
    "examine": 'adv/examine/',
    "wearItem": 'adv/wear/',
    "removeItem": 'adv/undress/',
    "nameChange": 'adv/change_name/',
    "pray": 'adv/pray/',
    "fly": 'adv/fly/',
    "dash": 'adv/dash/',
    "companionLetCarry": 'adv/carry/',
    "companionReceiveBack":  'adv/receive/',
    "lambdaCoinMine": 'bc/mine/'
    
}
header={'Authorization': 'Token ' + setupTable['token']}
init = setupTable['url'] + setupTable['init']
move = setupTable['url'] + setupTable['move']
take = setupTable['url'] + setupTable['take']
sell = setupTable['url'] + setupTable['sell']
drop = setupTable['url'] + setupTable['drop']
checkStatus = setupTable['url'] + setupTable['checkStatus']
examine = setupTable['url'] + setupTable['examine']
wearItem = setupTable['url'] + setupTable['wearItem']
removeItem = setupTable['url'] + setupTable['removeItem']
nameChange = setupTable['url'] + setupTable['nameChange']
pray = setupTable['url'] + setupTable['pray']
fly = setupTable['url'] + setupTable['fly']
dash = setupTable['url'] + setupTable['dash']
companionLetCarry = setupTable['url'] + setupTable['companionLetCarry']
companionReceiveBack = setupTable['url'] + setupTable['companionReceiveBack']
lambdaCoinMine = setupTable['url'] + setupTable['lambdaCoinMine']

In [169]:
class Player:
    
    def __init__(self):
        self.initResponse = requests.get(url=init, headers=header)
        self.playersMap = worldMap
        self.coolDown = None
        self.currentRoomID = None
        self.previousRoomID = None
        self.lastMoveDirection = None
        self.currentRoomInfo = None
        
        self.status = None
        self.inventory = None

        
    def getRoomInfo(self):
        time.sleep(1)
        response = requests.get(url=init, headers=header)
        
        if response.status_code == 200:
            return response.json()
        else:
            print(response.json())
            print(f'400 status code at getRoomInfo method {response.status_code}')
            
    def assignCurrentRoomInfo(self):
        self.currentRoomInfo = self.getRoomInfo()
        
    def assignCurrentRoomID(self):
        self.currentRoomID = self.getCurrentRoomID()
            
    def getCurrentRoomID(self):
        return self.currentRoomInfo['room_id']
            
    def writeNewRoomToMap(self):
        exits = self.currentRoomInfo['exits']
        
        if self.currentRoomID not in self.playersMap:
            self.playersMap[self.currentRoomID] = {key: '?' for key in exits}
            
    def writeMoveDirectionsForCurrentAndPreviousRoom(self, previousRoomID, currentRoomID, direction):
        self.playersMap[previousRoomID][direction] = self.currentRoomID
        self.playersMap[currentRoomID][inverseMap[direction]] = self.previousRoomID
        
    def moveRoom(self, direction):
        if str(self.currentRoomID) in self.playersMap and self.playersMap[str(self.currentRoomID)][direction] !='?':
            print("Wise Explore")
            
            self.previousRoomID = self.currentRoomID
            
            nextRoom = self.playersMap[str(self.currentRoomID)][direction]
            
            postRequestBody = {"direction":direction, "next_room_id":str(nextRoom)}
            response = requests.post(url=move, headers=header, json=postRequestBody)
            self.currentRoomInfo = response.json()
            self.coolDown = response.json()['cooldown']
            
            time.sleep(self.coolDown)
            self.currentRoomID = self.getCurrentRoomID()
            self.lastMoveDirection = direction
            
        else:
            print('Normal Explore')
            self.previousRoomID = self.currentRoomID
            
            postRequestBody = {"direction":direction}
            response = requests.post(url=move, headers=header, json=postRequestBody)

            self.currentRoomInfo = response.json()
            self.currentRoomID = self.getCurrentRoomID()
            
            if response.status_code == 200:
                self.coolDown = response.json()['cooldown']
                time.sleep(self.coolDown)
                
                self.lastMoveDirection = direction
                
                self.writeNewRoomToMap()
                self.writeMoveDirectionsForCurrentAndPreviousRoom(self.previousRoomID, self.currentRoomID, direction)
            else:
                print("The move was not a success")
                
        return self.getRoomInfo()
    
    def findUnexploredRoom(self):
        for direction in self.playersMap[self.currentRoomID]:
            if self.playersMap[self.currentRoomID][direction] == '?':
                return direction
        return direction
    
    def takeItem(self):
        itemName = self.currentRoomInfo['items']
        for items in itemName:
            postRequestBody = {"name": items}
            response = requests.post(url=take, headers=header, json=postRequestBody)
            
            self.coolDown = response.json()['cooldown']
            print("Cooling down for :"+ str(self.coolDown)+"seconds")
            time.sleep(self.coolDown)
        print("You have picked up ", itemName)
        
    def dropItem(self, itemName):
        pass
    
    def sellItem(self):
        self.checkPlayerStatus()
        time.sleep(2)
        if self.inventory:
            for item in self.inventory:
                postRequestBody = {"name": item, "confirm":"yes"}
                response = requests.post(url=sell, headers=header, json=postRequestBody)
                self.coolDown = response.json()['cooldown']
                time.sleep(self.coolDown)
                print("You have sold: ", item)
                
    def changeName(self):
        postRequestBody = {"name": "[CPHACHE]", "confirm":"aye"}
        response= requests.post(url=nameChange, headers=header, json=postRequestBody)
        
        print(response)
        print(response.json())
        
    
    def checkPlayerStatus(self):
        response = requests.post(url=checkStatus, headers=header)
        self.status = response.json()
        self.coolDown = response.json()['cooldown']
        
        if response.json()['inventory']:
            self.inventory = response.json()['inventory']
            
        time.sleep(self.coolDown)
        
        print(response.json())
        

In [170]:
def exploreWorld(player):
    print('Initializing...')
    
    roomStack = Stack()
    moveStack = Stack()
    count = 0
    

    player.currentRoomInfo = player.getRoomInfo()
    print('Loading currentRoomInfo')
    time.sleep(5)
    
    player.currentRoomID = player.currentRoomInfo['room_id']
    player.writeNewRoomToMap()
    
    roomStack.push(player.currentRoomID)
    
    roomDict = {}
    roomDict[player.currentRoomID] = player.currentRoomInfo
    
    while roomStack.size() > 0:
        
        direction = player.findUnexploredRoom()
        print("Direction", direction)
        
        if player.playersMap[player.currentRoomID][direction] == '?':
            
            
            player.moveRoom(direction)
            moveStack.push(direction)
            
            print("Leaving room: "+str(player.previousRoomID)+" entering room: "+str(player.currentRoomID))
            
            roomStack.push(player.currentRoomID)
            
            if player.currentRoomID not in roomDict:
                roomDict[player.currentRoomID] = player.currentRoomInfo
                
        else:
            print("roomStack:", roomStack.stack)
            print('moveStack:', moveStack.stack)
            roomStack.pop()
            lastMove = moveStack.stack[-1]
            backTrackingDirection = inverseMap[lastMove]
            
            if roomStack.size() > 0:
                player.previousRoomID = player.currentRoomID
                player.moveRoom(backTrackingDirection)
                player.lastMoveDirection = moveStack.stack[-1]      

            moveStack.pop()

    return roomDict          

In [171]:
def bfs(player, starting_vertex, destination_vertex):
    """
    Return a list containing the shortest path from
    starting_vertex to destination_vertex in
    breath-first order.
    """

    # Create an empty queue and enqueue the starting vertex ID
    pathQueue = Queue()
    directionQueue = Queue()
    
    pathQueue.enqueue([str(starting_vertex)])
    directionQueue.enqueue(['Starting Location'])

    # Create a Set to store visited vertices
    visited = set()

    # While the queue is not empty...
    while destination_vertex not in visited:
        # Dequeue the first vertex
        path = pathQueue.dequeue()
        directionPath = directionQueue.dequeue()

        # If that vertex has not been visited
        v = path[-1]
        
        if v not in visited:
            if v == destination_vertex:
                return path, directionPath
            visited.add(v)
            
        for direction, neighboringRoom in player.playersMap[v].items():
            path_copy = list(path)
            path_copy.append(str(neighboringRoom))
            pathQueue.enqueue(path_copy)
            
            directionCopy = list(directionPath)
            directionCopy.append(direction)
            directionQueue.enqueue(directionCopy)

    return None

In [172]:
def dfs(player, starting_vertex, destination_vertex):
    """
    Return a list containing the shortest path from
    starting_vertex to destination_vertex in
    breath-first order.
    """

    # Create an empty queue and enqueue the starting vertex ID
    pathStack = Stack()
    directionStack = Stack()
    
    pathStack.push([str(starting_vertex)])
    directionStack.push(['Starting Location'])

    # Create a Set to store visited vertices
    visited = set()

    # While the queue is not empty...
    while destination_vertex not in visited:
        # Dequeue the first vertex
        path = pathStack.pop()
        directionPath = directionStack.pop()

        # If that vertex has not been visited
        v = path[-1]
        
        if v not in visited:
            if v == destination_vertex:
                return path, directionPath
            visited.add(v)
            
        for direction, neighboringRoom in player.playersMap[v].items():
            path_copy = list(path)
            path_copy.append(str(neighboringRoom))
            pathStack.push(path_copy)
            
            directionCopy = list(directionPath)
            directionCopy.append(direction)
            directionStack.push(directionCopy)

    return None

In [173]:
def moveToSpecificRoom(player, starting_vertex, destination_vertex):
    path, directionPath = bfs(player, starting_vertex, destination_vertex)
    
    print(path)
    directions = directionPath[1:]
    print(directions)
    for direction in directions:
        player.moveRoom(direction)
        time.sleep(player.coolDown)
        
    print("You have reached your destination")

## Initialize Player

In [174]:
player = Player()
player.assignCurrentRoomInfo()
player.assignCurrentRoomID()

## World Points of Interest

Room 22 - Shrine
Room 461 - Shrine

## Everything below here is for exploration

In [62]:
player.takeItem()

Cooling down for :7.5seconds
Cooling down for :7.5seconds
Cooling down for :7.5seconds
You have picked up  ['shiny treasure', 'small treasure', 'tiny treasure']


In [91]:
player.moveRoom('n')

Wise Explore


{'room_id': 2,
 'title': 'A misty room',
 'description': 'You are standing on grass and surrounded by a dense mist. You can barely make out the exits in any direction.',
 'coordinates': '(60,59)',
 'elevation': 0,
 'terrain': 'NORMAL',
 'players': ['SirSnuffaluffagus'],
 'items': [],
 'exits': ['n', 's', 'e'],
 'cooldown': 1.0,
 'errors': [],
 'messages': []}

In [199]:
moveToSpecificRoom(player, player.currentRoomID, '100')

['467', '399', '362', '352', '349', '324', '298', '164', '156', '141', '112', '100']
['n', 'n', 'n', 'n', 'n', 'n', 'w', 'w', 'w', 'n', 'w']
Wise Explore
Wise Explore
Wise Explore
Wise Explore
Wise Explore
Wise Explore
Wise Explore
Wise Explore
Wise Explore
Wise Explore
Wise Explore
You have reached your destination


In [163]:
player.playersMap['461']

{'s': 141, 'e': 140, 'w': 100}

In [149]:
player.moveRoom('n')

Wise Explore


{'room_id': 100,
 'title': 'A misty room',
 'description': 'You are standing on grass and surrounded by a dense mist. You can barely make out the exits in any direction.',
 'coordinates': '(64,54)',
 'elevation': 0,
 'terrain': 'NORMAL',
 'players': [],
 'items': [],
 'exits': ['s', 'e', 'w'],
 'cooldown': 1.0,
 'errors': [],
 'messages': []}

In [175]:
player.changeName()

<Response [200]>
{'room_id': 467, 'title': "Pirate Ry's", 'description': "You see a sign before you that reads:\n\n'You have found Pirate Ry's. Send a `change_name` request and I'll change your identity to whatever you wish... for a price.'", 'coordinates': '(68,47)', 'elevation': 0, 'terrain': 'NORMAL', 'players': [], 'items': [], 'exits': ['n'], 'cooldown': 30.0, 'errors': [], 'messages': ['You have changed your name to [CPHACHE].', "'Ere's a tip from Pirate Ry: If you find a shrine, try prayin'. Ye' never know who may be listenin'."]}


In [176]:
import pandas as pd

df = pd.DataFrame(worldRoomIndex)

In [180]:
df = df.T

In [182]:
df.title.value_counts()

A misty room                427
A Dark Cave                  36
Mt. Holloway                 29
A brightly lit room           1
Wishing Well                  1
The Transmogriphier           1
Linh's Shrine                 1
Pirate Ry's                   1
The Peak of Mt. Holloway      1
Shop                          1
Glasowyn's Grave              1
Name: title, dtype: int64

In [194]:
dft = df[df['title'] != 'A misty room']
dfo = dft[dft['title']!= 'A Dark Cave']
dff = dfo[dfo['title']!='Mt. Holloway']

In [197]:
dff

,room_id,title,description,coordinates,elevation,terrain,players,items,exits,cooldown,errors,messages
467,467,Pirate Ry's,You see a sign before you that reads:\n\n'You ...,"(68,47)",0,NORMAL,[],[],[n],15,[],[You have walked south.]
22,22,The Peak of Mt. Holloway,You are standing at the zenith of Mt. Holloway...,"(61,55)",5,MOUNTAIN,[],[],"[n, s, w]",20,[],"[You have walked north., Uphill Penalty: 5s CD]"
0,0,A brightly lit room,You are standing in the center of a brightly l...,"(60,60)",0,NORMAL,"[player404, player405, player407, player408, p...",[],"[n, s, e, w]",15,[],[You have walked north.]
461,461,Linh's Shrine,You are standing before a glowing shrine to a ...,"(55,74)",0,CAVE,[],[tiny treasure],[s],15,[],[You have walked north.]
499,499,Glasowyn's Grave,You see a moss-topped gravestone which reads: ...,"(51,67)",0,NORMAL,[],[],[e],15,[],[You have walked west.]
55,55,Wishing Well,You are standing besides a large well. A sign ...,"(63,61)",0,NORMAL,[],[small treasure],[w],15,[],[You have walked east.]
1,1,Shop,You are standing in a small shop. A sign behin...,"(59,60)",0,NORMAL,"[player455, SirSnuffaluffagus, Fully, justinch...",[],[e],15,[],[You have walked west.]
495,495,The Transmogriphier,A strange machine stands in this room. There ...,"(50,58)",0,NORMAL,[],[tiny treasure],[e],15,[],[You have walked west.]
